* https://medium.com/analytics-vidhya/generating-a-dessert-ingredients-list-12edd1740753#id_token=eyJhbGciOiJSUzI1NiIsImtpZCI6ImFjYjZiZTUxZWZlYTZhNDE5ZWM5MzI1ZmVhYTFlYzQ2NjBmNWIzN2MiLCJ0eXAiOiJKV1QifQ.eyJpc3MiOiJodHRwczovL2FjY291bnRzLmdvb2dsZS5jb20iLCJuYmYiOjE2NDU3NTE3MjYsImF1ZCI6IjIxNjI5NjAzNTgzNC1rMWs2cWUwNjBzMnRwMmEyamFtNGxqZGNtczAwc3R0Zy5hcHBzLmdvb2dsZXVzZXJjb250ZW50LmNvbSIsInN1YiI6IjExMjg5ODkxMzg4OTk3MTcxMDQ0OSIsImVtYWlsIjoia2lya2VuZGFsbC5ldmVAZ21haWwuY29tIiwiZW1haWxfdmVyaWZpZWQiOnRydWUsImF6cCI6IjIxNjI5NjAzNTgzNC1rMWs2cWUwNjBzMnRwMmEyamFtNGxqZGNtczAwc3R0Zy5hcHBzLmdvb2dsZXVzZXJjb250ZW50LmNvbSIsIm5hbWUiOiJFdmVseW4gS2lya2VuZGFsbCIsInBpY3R1cmUiOiJodHRwczovL2xoMy5nb29nbGV1c2VyY29udGVudC5jb20vYS9BQVRYQUp5Y3lzVGVLUkdBOS1EcW40dnNsZWFZb0VfNnVkV1Q5WW9qcFBTMD1zOTYtYyIsImdpdmVuX25hbWUiOiJFdmVseW4iLCJmYW1pbHlfbmFtZSI6IktpcmtlbmRhbGwiLCJpYXQiOjE2NDU3NTIwMjYsImV4cCI6MTY0NTc1NTYyNiwianRpIjoiNzM3ZDEzNWU2NTk0YzZmNTZlMzNhNjJjYWUyNzZkMzk0ZmNkOWNmYiJ9.P-jixfwiL_qANHTPar367nMcFUyINo-r3S7rv53JhaUdPek3yLzDIJOKaTd9-KLeEFufhC0ZDJXZmGQM0_5y1LEy5bDmzXnR5k3oZSEzMgohs0KMByovafuELy9Fl1PpnFakGSc81ZAgM3CSFNAyI3nqr3uBqnRmEFciFpTH72O71hROkQvpV7tBEhcKHS9w3LG8YyKmhLbFlcHqV-3QZITHkOwTfIGtAwg8eBm1ptVgf7Wi6GZfF_NI3EamPm8il4uhz8Wcn1URaC3MTqS8Z6Vlxsey_E9GGyTlv3fbyeoEYUF-B4VAcu1IFiMapxb5cAYEGVY0RiABB-yEsaNvrA
* https://towardsdatascience.com/using-machine-learning-to-generate-recipes-that-actually-works-b2331c85ab72

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
cd /home/ec2-user/SageMaker/just-a-recipe-generator

/home/ec2-user/SageMaker/just-a-recipe-generator


In [ ]:
# !pip install tensorflow==2.8.0

In [4]:
import copy
import tensorflow as tf

In [10]:
from src.data.pickling import load_pickle
from src.features.clean_shoestring_data import clean_shoestring_recipes
from src.features.prepare_model_data import create_model_data

In [6]:
shoestring_recipes0 = load_pickle("data/raw/shoestring_recipes.pickle")

Loading data/raw/shoestring_recipes.pickle for consumption...


In [7]:
shoestring_recipes = copy.deepcopy(shoestring_recipes0)

In [8]:
recipe_dict = clean_shoestring_recipes(shoestring_recipes)

In [ ]:
recipe_dict

In [13]:
dataset0, dataset = create_model_data(recipe_dict)

print(type(dataset))
print(len(dataset))

<class 'tensorflow.python.data.ops.dataset_ops.PrefetchDataset'>
8


In [ ]:
# Generate recipe title first? And from that, generate ingredients?
# https://www.tensorflow.org/guide/keras/rnn#rnns_with_listdict_inputs_or_nested_inputs

## Create training batches

Before feeding this data into the model, you need to shuffle the data and pack it into batches.

## Build The Model
This section defines the model as a keras.Model subclass.

This model has three layers:

- tf.keras.layers.Embedding: The input layer. A trainable lookup table that will map each character-ID to a vector with embedding_dim dimensions;
- tf.keras.layers.GRU: A type of RNN with size units=rnn_units (You can also use an LSTM layer here.)
- tf.keras.layers.Dense: The output layer, with vocab_size outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

### References:

https://www.tensorflow.org/text/tutorials/text_generation#build_the_model

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units
)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

print(model.summary())

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

print(sampled_indices)

print("Input:\n", text_from_ids(input_example_batch[0]))
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices))